In [1]:
import pandas as pd
import numpy as np

# Generate synthetic network traffic data
np.random.seed(42)
time_index = pd.date_range(start="2024-01-01", periods=1000, freq="H")

data = pd.DataFrame({
    "timestamp": time_index,
    "network_traffic": np.random.randint(50, 500, size=(1000,)),  # Random traffic load
    "latency": np.random.uniform(10, 100, size=(1000,))  # Random latency in ms
})

data.to_csv("network_traffic.csv", index=False)
print("Sample data created successfully.")


ModuleNotFoundError: No module named 'pandas'

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Load dataset
data = pd.read_csv("network_traffic.csv")
data["timestamp"] = pd.to_datetime(data["timestamp"])
data.set_index("timestamp", inplace=True)

# Normalize data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Prepare data for LSTM
def create_sequences(data, seq_length=10):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i : i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(data_scaled)
X_train, y_train = X[:-100], y[:-100]
X_test, y_test = X[-100:], y[-100:]

# Define LSTM model
model = Sequential([
    LSTM(50, activation="relu", return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    LSTM(50, activation="relu"),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))
model.save("network_traffic_model.h5")

print("Model training complete.")


Epoch 1/20


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.2133 - val_loss: 0.0807
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0865 - val_loss: 0.0791
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0856 - val_loss: 0.0794
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0873 - val_loss: 0.0790
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0829 - val_loss: 0.0781
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0825 - val_loss: 0.0781
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0848 - val_loss: 0.0786
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0860 - val_loss: 0.0777
Epoch 9/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0856 - val_loss: 0.0775
Epoch 10/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0829 - val_loss: 0.0773
Epoch 11/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0836 - val_loss: 0.0779
Epoch 12/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0853 - val_loss: 0

Model training complete.


In [4]:
import gym
import numpy as np

class NetworkEnv(gym.Env):
    def __init__(self):
        self.state = np.random.randint(50, 500)  # Initial traffic load
        self.action_space = gym.spaces.Discrete(3)  # Increase, Decrease, Maintain
        self.observation_space = gym.spaces.Box(low=50, high=500, shape=(1,), dtype=np.int32)

    def step(self, action):
        if action == 0:
            self.state = max(50, self.state - 10)  # Reduce traffic load
        elif action == 1:
            self.state = min(500, self.state + 10)  # Increase traffic load

        reward = -abs(self.state - 250)  # Ideal load is 250
        done = False
        return np.array([self.state]), reward, done, {}

    def reset(self):
        self.state = np.random.randint(50, 500)
        return np.array([self.state])

env = NetworkEnv()


In [3]:
import stable_baselines3
print(stable_baselines3.__version__)


2.5.0


In [6]:
from stable_baselines3 import DQN

model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
model.save("bandwidth_optimizer")
print("Reinforcement Learning Model Saved.")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


a:\DISH5G\network_opt_env\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Reinforcement Learning Model Saved.
